# Email Data Cleaning Pipeline - Timezone-Aware Version

This notebook performs comprehensive data cleaning with:
1. Nanosecond precision handling in dates
2. **Timezone preservation and extraction**
3. **Timezone region classification**
4. Data quality validation
5. **Timezone-based grouping and analysis**

**Goal:** Preserve maximum data including timezone context for regional analysis

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

print("✅ Libraries loaded successfully")

✅ Libraries loaded successfully


In [2]:
# Load raw data with proper handling
print("="*80)
print("LOADING RAW DATA")
print("="*80)

# Try to load the data
file_path = "../data/processed/date_merge.csv"
df_raw = pd.read_csv(file_path)
original_count = len(df_raw)

print(f"\n📊 Dataset loaded:")
print(f"  • Rows: {original_count:,}")
print(f"  • Columns: {len(df_raw.columns)}")
print(f"  • Column names: {df_raw.columns.tolist()}")

# Check if date column needs special handling
print("\n🔍 Checking date column format...")
sample_date = str(df_raw['date'].iloc[0])
print(f"  Sample date: {sample_date}")

# Check for nanosecond precision issue
if '.000000000' in sample_date or 'T' in sample_date:
    print("  ⚠️ Detected datetime with nanosecond precision or ISO format")
    print("  ✅ Will apply special handling...")

# Check data types
print("\n📋 Column data types:")
for col, dtype in df_raw.dtypes.items():
    null_count = df_raw[col].isna().sum()
    print(f"  • {col}: {dtype} (nulls: {null_count})")

LOADING RAW DATA

📊 Dataset loaded:
  • Rows: 49,301
  • Columns: 7
  • Column names: ['sender', 'receiver', 'date', 'subject', 'body', 'label', 'urls']

🔍 Checking date column format...
  Sample date: Tue, 05 Aug 2008 16:31:02 -0700
  ⚠️ Detected datetime with nanosecond precision or ISO format
  ✅ Will apply special handling...

📋 Column data types:
  • sender: object (nulls: 0)
  • receiver: object (nulls: 0)
  • date: object (nulls: 0)
  • subject: object (nulls: 77)
  • body: object (nulls: 1)
  • label: int64 (nulls: 0)
  • urls: int64 (nulls: 0)


In [3]:
# Define timezone to region mapping
print("="*80)
print("TIMEZONE REGION MAPPING SETUP")
print("="*80)

def get_simple_region(offset_hours):
    """
    Simplified 6-region model for cleaner visualization
    Valid range: UTC-12:00 to UTC+14:00
    
    Boundary handling:
    - UTC-4 belongs to Americas (e.g., Atlantic Time)
    - UTC+2 belongs to Europe/Africa (e.g., South Africa)
    - UTC+6 belongs to Middle East/South Asia (e.g., Bangladesh)
    - UTC+10 belongs to APAC (e.g., Australian Eastern Time)
    """
    if pd.isna(offset_hours):
        return 'Unknown'
    elif -12 <= offset_hours < -4:
        return 'Americas'
    elif -4 <= offset_hours <= 2:
        return 'Europe/Africa'
    elif 2 < offset_hours <= 6:
        return 'Middle East/South Asia'  # Includes all .5 offsets (Iran, India, etc.)
    elif 6 < offset_hours <= 10:
        return 'APAC'  # Includes Myanmar (6.5), Australia Central (9.5)
    elif 10 < offset_hours <= 14:
        return 'Oceania/Pacific'  # Includes Australia (10.5)
    else:
        return 'Invalid'

# Test the function with boundary values
print("\n🌍 Sample timezone region mappings:")
test_offsets = [-12, -8, -5, -4, -3, 0, 2, 3, 5.5, 6, 6.5, 8, 9.5, 10, 10.5, 12, 14]
for offset in test_offsets:
    region = get_simple_region(offset)
    print(f"  UTC{offset:+.1f} → {region}")

print("\n✅ Timezone mapping function ready")
print("  • Valid range: UTC-12:00 to UTC+14:00")
print("  • Data outside this range will be removed")

TIMEZONE REGION MAPPING SETUP

🌍 Sample timezone region mappings:
  UTC-12.0 → Americas
  UTC-8.0 → Americas
  UTC-5.0 → Americas
  UTC-4.0 → Europe/Africa
  UTC-3.0 → Europe/Africa
  UTC+0.0 → Europe/Africa
  UTC+2.0 → Europe/Africa
  UTC+3.0 → Middle East/South Asia
  UTC+5.5 → Middle East/South Asia
  UTC+6.0 → Middle East/South Asia
  UTC+6.5 → APAC
  UTC+8.0 → APAC
  UTC+9.5 → APAC
  UTC+10.0 → APAC
  UTC+10.5 → Oceania/Pacific
  UTC+12.0 → Oceania/Pacific
  UTC+14.0 → Oceania/Pacific

✅ Timezone mapping function ready
  • Valid range: UTC-12:00 to UTC+14:00
  • Data outside this range will be removed


In [4]:
# Enhanced Date Parsing with Timezone Preservation
print("="*80)
print("DATE PARSING WITH TIMEZONE PRESERVATION")
print("="*80)

# Step 1: Extract timezone information
print("\n📅 Step 1: Extracting timezone information...")

# Extract timezone offset from date string
df_raw['timezone_offset'] = df_raw['date'].astype(str).str.extract(r'([+-]\d{4})$')[0]

# Count timezones
has_timezone = df_raw['timezone_offset'].notna().sum()
no_timezone = df_raw['timezone_offset'].isna().sum()

print(f"  • Dates with timezone: {has_timezone:,}")
print(f"  • Dates without timezone: {no_timezone:,}")

if no_timezone > 0:
    print(f"  ℹ️  Missing timezones will be treated as UTC (+0000)")

# Convert timezone offset to hours for easier handling
def parse_timezone_offset(tz_str):
    """Convert timezone string like '+0800' or '-0700' to hours"""
    if pd.isna(tz_str):
        # Treat missing timezone as UTC (+0000)
        return 0.0
    try:
        sign = 1 if tz_str[0] == '+' else -1
        hours = int(tz_str[1:3])
        minutes = int(tz_str[3:5])
        return sign * (hours + minutes / 60)
    except:
        # If parsing fails, default to UTC
        return 0.0

df_raw['timezone_hours'] = df_raw['timezone_offset'].apply(parse_timezone_offset)

# Remove rows with timezones outside valid range (UTC-12 to UTC+14)
print("\n🔍 Step 2: Validating timezone ranges...")
valid_tz_mask = (df_raw['timezone_hours'] >= -12) & (df_raw['timezone_hours'] <= 14)
invalid_tz_count = (~valid_tz_mask).sum()

if invalid_tz_count > 0:
    print(f"  ⚠️  Found {invalid_tz_count} emails with invalid timezones (outside UTC-12 to UTC+14)")
    print(f"  🗑️  Removing these {invalid_tz_count} rows from the dataset")
    df_raw = df_raw[valid_tz_mask].copy()
    print(f"  ✅ Kept {len(df_raw):,} rows with valid timezones")
else:
    print(f"  ✅ All timezones are valid (within UTC-12 to UTC+14)")

# Step 3: Add timezone region using simplified mapping
print("\n🌍 Step 3: Classifying timezone regions...")
df_raw['timezone_region'] = df_raw['timezone_hours'].apply(get_simple_region)

# Verify no 'Invalid' regions were created (safety check)
invalid_regions = (df_raw['timezone_region'] == 'Invalid').sum()
if invalid_regions > 0:
    print(f"  ⚠️  WARNING: Found {invalid_regions} emails classified as 'Invalid' region")
    print(f"  🗑️  Removing these rows")
    df_raw = df_raw[df_raw['timezone_region'] != 'Invalid'].copy()

# Show distribution
print("\n  Timezone region distribution:")
region_counts = df_raw['timezone_region'].value_counts()
for region, count in region_counts.items():
    pct = count / len(df_raw) * 100
    print(f"    • {region}: {count:,} ({pct:.1f}%)")

# Step 4: Parse dates while preserving timezone
print("\n📅 Step 4: Parsing dates (keeping original timezone context)...")

def parse_email_date_preserve_tz(date_str):
    """
    Parse email date string and return UTC datetime
    (but we keep timezone info in separate columns)
    """
    try:
        # Try standard email format with day name
        # Example: "Tue, 05 Aug 2008 16:31:02 -0700"
        from email.utils import parsedate_to_datetime
        dt = parsedate_to_datetime(date_str)
        # Convert to UTC (timezone-naive) for consistent storage
        return dt.replace(tzinfo=None) if dt.tzinfo is None else dt.astimezone(None).replace(tzinfo=None)
    except:
        pass
    
    try:
        # Fallback: try without day name
        cleaned = re.sub(r'^\w{3},\s*', '', str(date_str))
        dt = parsedate_to_datetime(cleaned)
        return dt.replace(tzinfo=None) if dt.tzinfo is None else dt.astimezone(None).replace(tzinfo=None)
    except:
        return pd.NaT

# Parse dates
df_raw['date_utc'] = df_raw['date'].apply(parse_email_date_preserve_tz)

# ✅ FIX: Explicitly convert to datetime type
df_raw['date_utc'] = pd.to_datetime(df_raw['date_utc'], errors='coerce')

# Report parsing success
parsed_count = df_raw['date_utc'].notna().sum()
failed_count = df_raw['date_utc'].isna().sum()

print(f"  • Successfully parsed: {parsed_count:,}")
print(f"  • Failed to parse: {failed_count:,}")

if failed_count > 0:
    print(f"\n  ⚠️ Removing {failed_count:,} rows with unparseable dates")
    df_raw = df_raw[df_raw['date_utc'].notna()].copy()

# Rename for clarity
df_raw = df_raw.rename(columns={'date': 'date_original', 'date_utc': 'date'})

# Verify the date column is now datetime type
print(f"\n✅ Date column type: {df_raw['date'].dtype}")
print("\n✅ Date parsing complete with timezone preservation!")

DATE PARSING WITH TIMEZONE PRESERVATION

📅 Step 1: Extracting timezone information...
  • Dates with timezone: 48,242
  • Dates without timezone: 1,059
  ℹ️  Missing timezones will be treated as UTC (+0000)

🔍 Step 2: Validating timezone ranges...
  ⚠️  Found 388 emails with invalid timezones (outside UTC-12 to UTC+14)
  🗑️  Removing these 388 rows from the dataset
  ✅ Kept 48,913 rows with valid timezones

🌍 Step 3: Classifying timezone regions...

  Timezone region distribution:
    • Europe/Africa: 27,500 (56.2%)
    • Americas: 12,778 (26.1%)
    • Middle East/South Asia: 4,785 (9.8%)
    • APAC: 2,857 (5.8%)
    • Oceania/Pacific: 993 (2.0%)

📅 Step 4: Parsing dates (keeping original timezone context)...
  • Successfully parsed: 48,836
  • Failed to parse: 77

  ⚠️ Removing 77 rows with unparseable dates

✅ Date column type: datetime64[ns]

✅ Date parsing complete with timezone preservation!


In [5]:
# Validate and Clean Date Anomalies
print("="*80)
print("DATE VALIDATION AND ANOMALY DETECTION")
print("="*80)

# Define reasonable date range
min_valid_date = pd.Timestamp('1990-01-01')
max_valid_date = pd.Timestamp('2025-12-31')

print(f"\n📅 Valid date range: {min_valid_date.date()} to {max_valid_date.date()}")

# Check for dates outside valid range
too_old = (df_raw['date'] < min_valid_date).sum()
too_new = (df_raw['date'] > max_valid_date).sum()

print(f"\n🔍 Date anomalies:")
print(f"  • Dates before {min_valid_date.date()}: {too_old:,}")
print(f"  • Dates after {max_valid_date.date()}: {too_new:,}")

if too_old > 0 or too_new > 0:
    print(f"\n  ⚠️ Filtering out {too_old + too_new:,} anomalous dates")
    df_raw = df_raw[
        (df_raw['date'] >= min_valid_date) & 
        (df_raw['date'] <= max_valid_date)
    ].copy()

# Show final date statistics
print(f"\n📊 Final date statistics:")
print(f"  • Earliest date: {df_raw['date'].min()}")
print(f"  • Latest date: {df_raw['date'].max()}")
print(f"  • Date span: {(df_raw['date'].max() - df_raw['date'].min()).days:,} days")
print(f"  • Unique dates: {df_raw['date'].nunique():,}")

print("\n✅ Date validation complete")

DATE VALIDATION AND ANOMALY DETECTION

📅 Valid date range: 1990-01-01 to 2025-12-31

🔍 Date anomalies:
  • Dates before 1990-01-01: 18
  • Dates after 2025-12-31: 9

  ⚠️ Filtering out 27 anomalous dates

📊 Final date statistics:
  • Earliest date: 1990-01-01 17:57:46
  • Latest date: 2022-12-27 10:56:49
  • Date span: 12,047 days
  • Unique dates: 45,189

✅ Date validation complete


In [6]:
# Extract Temporal Features for Analysis
print("="*80)
print("EXTRACTING TEMPORAL FEATURES")
print("="*80)

# Extract useful temporal components
df_raw['year'] = df_raw['date'].dt.year
df_raw['month'] = df_raw['date'].dt.month
df_raw['day'] = df_raw['date'].dt.day
df_raw['hour'] = df_raw['date'].dt.hour
df_raw['day_of_week'] = df_raw['date'].dt.dayofweek  # 0=Monday, 6=Sunday
df_raw['day_name'] = df_raw['date'].dt.day_name()
df_raw['is_weekend'] = df_raw['day_of_week'].isin([5, 6]).astype(int)

print("\n✅ Extracted temporal features:")
print("  • year, month, day, hour")
print("  • day_of_week, day_name, is_weekend")

# Show sample
print("\n📋 Sample of temporal features:")
print(df_raw[['date', 'year', 'month', 'hour', 'day_name', 'is_weekend', 'timezone_hours', 'timezone_region']].head(10))

EXTRACTING TEMPORAL FEATURES

✅ Extracted temporal features:
  • year, month, day, hour
  • day_of_week, day_name, is_weekend

📋 Sample of temporal features:
                 date  year  month  hour   day_name  is_weekend  \
0 2008-08-06 07:31:02  2008      8     7  Wednesday           0   
1 2008-08-06 07:31:03  2008      8     7  Wednesday           0   
2 2008-08-06 16:28:00  2008      8    16  Wednesday           0   
3 2008-08-06 07:31:20  2008      8     7  Wednesday           0   
4 2008-08-06 07:31:21  2008      8     7  Wednesday           0   
5 2008-08-06 07:31:22  2008      8     7  Wednesday           0   
6 2008-08-06 05:44:01  2008      8     5  Wednesday           0   
7 2008-08-06 07:41:14  2008      8     7  Wednesday           0   
8 2008-08-06 07:31:03  2008      8     7  Wednesday           0   
9 2008-08-06 07:31:38  2008      8     7  Wednesday           0   

   timezone_hours         timezone_region  
0           -7.00                Americas  
1           -5.0

In [7]:
# Final Data Quality Check
print("="*80)
print("FINAL DATA QUALITY CHECK")
print("="*80)

# Check for missing values
print("\n📊 Missing Values:")
for col in df_raw.columns:
    missing = df_raw[col].isna().sum()
    if missing > 0:
        missing_pct = missing / len(df_raw) * 100
        print(f"  • {col}: {missing:,} ({missing_pct:.1f}%)")

# Check for duplicates
duplicates = df_raw.duplicated(subset=['sender', 'receiver', 'date', 'subject']).sum()
print(f"\n📊 Duplicate rows: {duplicates:,}")
if duplicates > 0:
    print("  Removing duplicates...")
    df_raw = df_raw.drop_duplicates(subset=['sender', 'receiver', 'date', 'subject']).copy()
    print(f"  ✅ Removed {duplicates:,} duplicate rows")

# Validate email formats
print("\n📊 Email Format Validation:")
has_email_format = df_raw['sender'].str.contains(r'[<>@]', na=False).sum()
print(f"  Senders with email format: {has_email_format:,} ({has_email_format/len(df_raw)*100:.1f}%)")

# Final data types check
print("\n📊 Final Data Types:")
key_columns = ['sender', 'receiver', 'date', 'subject', 'label', 'timezone_hours', 'timezone_region']
for col in key_columns:
    if col in df_raw.columns:
        print(f"  • {col}: {df_raw[col].dtype}")

print(f"\n📊 Final Dataset Size:")
print(f"  • Rows: {len(df_raw):,}")
print(f"  • Columns: {len(df_raw.columns)}")
print(f"  • Date range: {df_raw['date'].min()} to {df_raw['date'].max()}")

FINAL DATA QUALITY CHECK

📊 Missing Values:
  • subject: 77 (0.2%)
  • body: 1 (0.0%)
  • timezone_offset: 1,038 (2.1%)

📊 Duplicate rows: 6
  Removing duplicates...
  ✅ Removed 6 duplicate rows

📊 Email Format Validation:
  Senders with email format: 48,786 (100.0%)

📊 Final Data Types:
  • sender: object
  • receiver: object
  • date: datetime64[ns]
  • subject: object
  • label: int64
  • timezone_hours: float64
  • timezone_region: object

📊 Final Dataset Size:
  • Rows: 48,803
  • Columns: 18
  • Date range: 1990-01-01 17:57:46 to 2022-12-27 10:56:49


In [8]:
# Timezone-Based Analysis
print("="*80)
print("TIMEZONE-BASED ANALYSIS")
print("="*80)

# 1. Overall distribution by timezone region
print("\n📊 1. EMAIL DISTRIBUTION BY TIMEZONE REGION")
print("="*80)

region_stats = df_raw.groupby('timezone_region').agg({
    'label': ['count', 'sum', 'mean']
}).round(3)

region_stats.columns = ['Total_Emails', 'Phishing_Count', 'Phishing_Rate']
region_stats['Legitimate_Count'] = region_stats['Total_Emails'] - region_stats['Phishing_Count']
region_stats = region_stats.sort_values('Total_Emails', ascending=False)

print("\n" + region_stats.to_string())

# 2. Phishing vs Legitimate by timezone region
print("\n\n📊 2. PHISHING VS LEGITIMATE BY REGION")
print("="*80)

for region in region_stats.index:
    region_data = region_stats.loc[region]
    total = region_data['Total_Emails']
    phishing = region_data['Phishing_Count']
    legitimate = region_data['Legitimate_Count']
    phishing_pct = region_data['Phishing_Rate'] * 100
    
    print(f"\n{region}:")
    print(f"  Total: {int(total):,} emails")
    print(f"  Phishing: {int(phishing):,} ({phishing_pct:.1f}%)")
    print(f"  Legitimate: {int(legitimate):,} ({100-phishing_pct:.1f}%)")

# 3. Top timezone hours (numeric offsets)
print("\n\n📊 3. TOP 10 TIMEZONE HOURS (NUMERIC OFFSET)")
print("="*80)

tz_hours_stats = df_raw.groupby('timezone_hours').agg({
    'label': ['count', 'sum']
}).round(0)

tz_hours_stats.columns = ['Total_Emails', 'Phishing_Count']
tz_hours_stats['Phishing_Rate'] = (tz_hours_stats['Phishing_Count'] / tz_hours_stats['Total_Emails'] * 100).round(1)
tz_hours_stats = tz_hours_stats.sort_values('Total_Emails', ascending=False).head(10)

print("\n" + tz_hours_stats.to_string())

# 4. Temporal patterns by timezone region
print("\n\n📊 4. TEMPORAL PATTERNS BY REGION")
print("="*80)

# Hour of day distribution by region
print("\nMost active hours by region (Top 3):")
for region in df_raw['timezone_region'].value_counts().head(5).index:
    if region not in ['Unknown', 'Invalid']:
        region_data = df_raw[df_raw['timezone_region'] == region]
        top_hours = region_data['hour'].value_counts().head(3)
        print(f"\n{region}:")
        for hour, count in top_hours.items():
            print(f"  {hour:02d}:00 - {count:,} emails")

# 5. Yearly trends by timezone region
print("\n\n📊 5. YEARLY TRENDS BY REGION (2008-2022)")
print("="*80)

yearly_region = df_raw.groupby(['year', 'timezone_region']).size().unstack(fill_value=0)

# Show years with most emails
top_years = df_raw['year'].value_counts().head(5).index
print("\nEmails per year (top 5 years):")
for year in sorted(top_years):
    if year in yearly_region.index:
        year_total = yearly_region.loc[year].sum()
        print(f"\n{year}: {int(year_total):,} total emails")
        for region in yearly_region.columns:
            count = yearly_region.loc[year, region]
            if count > 0:
                pct = count / year_total * 100
                print(f"  {region}: {int(count):,} ({pct:.1f}%)")

TIMEZONE-BASED ANALYSIS

📊 1. EMAIL DISTRIBUTION BY TIMEZONE REGION

                        Total_Emails  Phishing_Count  Phishing_Rate  Legitimate_Count
timezone_region                                                                      
Europe/Africa                  27446           16379           0.60             11067
Americas                       12751            4328           0.34              8423
Middle East/South Asia          4776            4453           0.93               323
APAC                            2844            2175           0.77               669
Oceania/Pacific                  986              71           0.07               915


📊 2. PHISHING VS LEGITIMATE BY REGION

Europe/Africa:
  Total: 27,446 emails
  Phishing: 16,379 (59.7%)
  Legitimate: 11,067 (40.3%)

Americas:
  Total: 12,751 emails
  Phishing: 4,328 (33.9%)
  Legitimate: 8,423 (66.1%)

Middle East/South Asia:
  Total: 4,776 emails
  Phishing: 4,453 (93.2%)
  Legitimate: 323 (6.8%)

APAC:
 

In [9]:
# Email Distribution by Timezone Hours (Detailed)
print("="*80)
print("EMAIL DISTRIBUTION BY TIMEZONE HOURS")
print("="*80)

# Group by timezone_hours and calculate statistics
tz_hours_distribution = df_raw.groupby('timezone_hours').agg({
    'label': ['count', 'sum', 'mean']
}).round(3)

tz_hours_distribution.columns = ['Total_Emails', 'Phishing_Count', 'Phishing_Rate']
tz_hours_distribution['Legitimate_Count'] = tz_hours_distribution['Total_Emails'] - tz_hours_distribution['Phishing_Count']
tz_hours_distribution['Phishing_Percentage'] = (tz_hours_distribution['Phishing_Rate'] * 100).round(1)

# Sort by timezone hours (ascending)
tz_hours_distribution = tz_hours_distribution.sort_index()

print(f"\n📊 Total unique timezone offsets: {len(tz_hours_distribution)}")
print(f"\n{'Timezone':<12} {'Total':<10} {'Phishing':<10} {'Legitimate':<12} {'Phishing %':<12}")
print("="*80)

for tz_hour, row in tz_hours_distribution.iterrows():
    if pd.isna(tz_hour):
        tz_label = "Unknown"
    else:
        tz_label = f"UTC{tz_hour:+.1f}"
    
    total = int(row['Total_Emails'])
    phishing = int(row['Phishing_Count'])
    legitimate = int(row['Legitimate_Count'])
    phishing_pct = row['Phishing_Percentage']
    
    print(f"{tz_label:<12} {total:<10,} {phishing:<10,} {legitimate:<12,} {phishing_pct:<12.1f}%")

# Show top 10 by volume
print("\n\n📊 TOP 10 TIMEZONE HOURS BY EMAIL VOLUME")
print("="*80)

top_10_tz = tz_hours_distribution.sort_values('Total_Emails', ascending=False).head(10)

print(f"\n{'Rank':<6} {'Timezone':<12} {'Total':<10} {'Phishing':<10} {'Legitimate':<12} {'Phishing %':<12}")
print("="*80)

for rank, (tz_hour, row) in enumerate(top_10_tz.iterrows(), 1):
    if pd.isna(tz_hour):
        tz_label = "Unknown"
    else:
        tz_label = f"UTC{tz_hour:+.1f}"
    
    total = int(row['Total_Emails'])
    phishing = int(row['Phishing_Count'])
    legitimate = int(row['Legitimate_Count'])
    phishing_pct = row['Phishing_Percentage']
    
    print(f"{rank:<6} {tz_label:<12} {total:<10,} {phishing:<10,} {legitimate:<12,} {phishing_pct:<12.1f}%")

# Show timezone hours with highest phishing rates (min 100 emails)
print("\n\n📊 TIMEZONE HOURS WITH HIGHEST PHISHING RATES (min 100 emails)")
print("="*80)

high_phishing_tz = tz_hours_distribution[tz_hours_distribution['Total_Emails'] >= 100].sort_values('Phishing_Rate', ascending=False).head(10)

print(f"\n{'Rank':<6} {'Timezone':<12} {'Total':<10} {'Phishing':<10} {'Phishing %':<12}")
print("="*80)

for rank, (tz_hour, row) in enumerate(high_phishing_tz.iterrows(), 1):
    if pd.isna(tz_hour):
        tz_label = "Unknown"
    else:
        tz_label = f"UTC{tz_hour:+.1f}"
    
    total = int(row['Total_Emails'])
    phishing = int(row['Phishing_Count'])
    phishing_pct = row['Phishing_Percentage']
    
    print(f"{rank:<6} {tz_label:<12} {total:<10,} {phishing:<10,} {phishing_pct:<12.1f}%")

# Show timezone hours with lowest phishing rates (min 100 emails)
print("\n\n📊 TIMEZONE HOURS WITH LOWEST PHISHING RATES (min 100 emails)")
print("="*80)

low_phishing_tz = tz_hours_distribution[tz_hours_distribution['Total_Emails'] >= 100].sort_values('Phishing_Rate', ascending=True).head(10)

print(f"\n{'Rank':<6} {'Timezone':<12} {'Total':<10} {'Legitimate':<12} {'Phishing %':<12}")
print("="*80)

for rank, (tz_hour, row) in enumerate(low_phishing_tz.iterrows(), 1):
    if pd.isna(tz_hour):
        tz_label = "Unknown"
    else:
        tz_label = f"UTC{tz_hour:+.1f}"
    
    total = int(row['Total_Emails'])
    legitimate = int(row['Legitimate_Count'])
    phishing_pct = row['Phishing_Percentage']
    
    print(f"{rank:<6} {tz_label:<12} {total:<10,} {legitimate:<12,} {phishing_pct:<12.1f}%")

print("\n✅ Timezone hours distribution analysis complete!")

EMAIL DISTRIBUTION BY TIMEZONE HOURS

📊 Total unique timezone offsets: 40

Timezone     Total      Phishing   Legitimate   Phishing %  
UTC-12.0     44         44         0            100.0       %
UTC-11.0     11         11         0            100.0       %
UTC-10.0     74         68         6            91.9        %
UTC-9.5      4          4          0            100.0       %
UTC-9.0      55         53         2            96.4        %
UTC-8.0      2,435      590        1,845        24.2        %
UTC-7.0      3,966      1,165      2,801        29.4        %
UTC-6.5      1          1          0            100.0       %
UTC-6.0      1,133      413        720          36.5        %
UTC-5.5      4          4          0            100.0       %
UTC-5.0      5,003      1,954      3,049        39.1        %
UTC-4.5      21         21         0            100.0       %
UTC-4.0      5,017      3,275      1,742        65.3        %
UTC-3.5      1          1          0            100.0     

In [10]:
# Weekend vs Weekday Analysis by Timezone
print("="*80)
print("WEEKEND VS WEEKDAY ANALYSIS BY TIMEZONE")
print("="*80)

weekend_analysis = df_raw.groupby(['timezone_region', 'is_weekend']).agg({
    'label': ['count', 'sum', 'mean']
}).round(3)

weekend_analysis.columns = ['Total_Emails', 'Phishing_Count', 'Phishing_Rate']

print("\nEmails sent during weekends vs weekdays by region:")
print("\n(0 = Weekday, 1 = Weekend)\n")

for region in df_raw['timezone_region'].value_counts().head(5).index:
    if region in weekend_analysis.index.get_level_values(0):
        print(f"\n{region}:")
        region_data = weekend_analysis.loc[region]
        
        for is_weekend, row in region_data.iterrows():
            day_type = "Weekend" if is_weekend == 1 else "Weekday"
            total = row['Total_Emails']
            phishing = row['Phishing_Count']
            rate = row['Phishing_Rate'] * 100
            
            print(f"  {day_type}: {int(total):,} emails, {int(phishing):,} phishing ({rate:.1f}%)")

WEEKEND VS WEEKDAY ANALYSIS BY TIMEZONE

Emails sent during weekends vs weekdays by region:

(0 = Weekday, 1 = Weekend)


Europe/Africa:
  Weekday: 26,071 emails, 15,468 phishing (59.3%)
  Weekend: 1,375 emails, 911 phishing (66.3%)

Americas:
  Weekday: 12,176 emails, 3,943 phishing (32.4%)
  Weekend: 575 emails, 385 phishing (67.0%)

Middle East/South Asia:
  Weekday: 4,702 emails, 4,386 phishing (93.3%)
  Weekend: 74 emails, 67 phishing (90.5%)

APAC:
  Weekday: 2,736 emails, 2,081 phishing (76.1%)
  Weekend: 108 emails, 94 phishing (87.0%)

Oceania/Pacific:
  Weekday: 980 emails, 71 phishing (7.2%)
  Weekend: 6 emails, 0 phishing (0.0%)


In [11]:
# Save cleaned data with timezone information
print("="*80)
print("SAVING CLEANED DATA WITH TIMEZONE INFORMATION")
print("="*80)

# Prepare final dataset
essential_cols = [
    'sender', 'receiver', 'date', 'subject', 'body', 'label', 'urls',
    'timezone_hours', 'timezone_region',
    'year', 'month', 'day', 'hour', 'day_of_week', 'day_name', 'is_weekend'
]

# Keep only columns that exist
final_cols = [col for col in essential_cols if col in df_raw.columns]
df_final = df_raw[final_cols].copy()

# Format date to avoid nanosecond issues when saving
df_final['date'] = df_final['date'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Save to CSV
output_path = "cleaned_date_merged_data.csv"
df_final.to_csv(output_path, index=False)

print(f"\n✅ Data saved to: {output_path}")
print(f"  • Rows: {len(df_final):,}")
print(f"  • Columns: {len(df_final.columns)}")
print(f"  • New columns added:")
print(f"    - timezone_hours (numeric offset in hours, e.g., 8.0, -7.0)")
print(f"    - timezone_region (geographic classification)")
print(f"    - year, month, day, hour, day_of_week, day_name, is_weekend")

# Convert date back to datetime for immediate use
df_final['date'] = pd.to_datetime(df_final['date'])

# Summary statistics
print("\n📊 FINAL SUMMARY:")
print("="*80)
print(f"  Started with: {original_count:,} rows")
print(f"  Ended with: {len(df_final):,} rows")
print(f"  Data retention: {len(df_final)/original_count*100:.2f}%")
print(f"  Date range: {df_final['date'].min()} to {df_final['date'].max()}")
print(f"  Time span: {(df_final['date'].max() - df_final['date'].min()).days:,} days")

# Label distribution
phishing = (df_final['label'] == 1).sum()
legitimate = (df_final['label'] == 0).sum()
print(f"\n  Label distribution:")
print(f"    • Phishing: {phishing:,} ({phishing/len(df_final)*100:.1f}%)")
print(f"    • Legitimate: {legitimate:,} ({legitimate/len(df_final)*100:.1f}%)")

# Timezone distribution
print(f"\n  Timezone regions:")
for region, count in df_final['timezone_region'].value_counts().head(5).items():
    pct = count / len(df_final) * 100
    print(f"    • {region}: {count:,} ({pct:.1f}%)")

print("\n" + "="*80)
print("✅ DATA CLEANING COMPLETE WITH TIMEZONE ANALYSIS!")
print("="*80)

# Return the cleaned dataframe for further use
df_cleaned = df_final.copy()

SAVING CLEANED DATA WITH TIMEZONE INFORMATION

✅ Data saved to: cleaned_date_merged_data.csv
  • Rows: 48,803
  • Columns: 16
  • New columns added:
    - timezone_hours (numeric offset in hours, e.g., 8.0, -7.0)
    - timezone_region (geographic classification)
    - year, month, day, hour, day_of_week, day_name, is_weekend

📊 FINAL SUMMARY:
  Started with: 49,301 rows
  Ended with: 48,803 rows
  Data retention: 98.99%
  Date range: 1990-01-01 17:57:46 to 2022-12-27 10:56:49
  Time span: 12,047 days

  Label distribution:
    • Phishing: 27,406 (56.2%)
    • Legitimate: 21,397 (43.8%)

  Timezone regions:
    • Europe/Africa: 27,446 (56.2%)
    • Americas: 12,751 (26.1%)
    • Middle East/South Asia: 4,776 (9.8%)
    • APAC: 2,844 (5.8%)
    • Oceania/Pacific: 986 (2.0%)

✅ DATA CLEANING COMPLETE WITH TIMEZONE ANALYSIS!


In [12]:
# Instructions for loading the cleaned data
print("="*80)
print("HOW TO LOAD THE CLEANED DATA IN OTHER NOTEBOOKS")
print("="*80)

print("""
To load this cleaned data with timezone information in your analysis notebooks, use:

```python
import pandas as pd

# Load the cleaned data
df = pd.read_csv('cleaned_date_merged_data.csv')

# Parse the date column
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')

# Verify
print(f"Loaded {len(df):,} rows")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")
print(f"Timezone regions: {df['timezone_region'].unique()}")

# Example: Group by timezone region
region_analysis = df.groupby('timezone_region')['label'].agg(['count', 'sum', 'mean'])
print("\\nPhishing rates by region:")
print(region_analysis)

# Example: Filter by specific region
asia_emails = df[df['timezone_region'] == 'APAC']
print(f"\\nEmails from APAC: {len(asia_emails):,}")

# Example: Filter by timezone hours (numeric comparisons)
east_asian_tz = df[(df['timezone_hours'] >= 7) & (df['timezone_hours'] <= 9)]
print(f"\\nEmails from UTC+7 to UTC+9: {len(east_asian_tz):,}")

# Example: Analyze by timezone hours
tz_analysis = df.groupby('timezone_hours')['label'].agg(['count', 'mean'])
print("\\nTop timezones by volume:")
print(tz_analysis.sort_values('count', ascending=False).head(10))
```

New columns available:
- timezone_hours: Numeric timezone offset in hours (e.g., 8.0, -7.0) - useful for filtering and analysis
- timezone_region: Geographic region classification (Americas, Europe/Africa, Middle East/South Asia, APAC, Oceania/Pacific)
- year, month, day, hour: Temporal components extracted from date
- day_of_week, day_name: Day information (0=Monday, 6=Sunday)
- is_weekend: Binary indicator (1=weekend, 0=weekday)
""")

print("\n✅ Your data is clean and ready for timezone-based analysis!")

HOW TO LOAD THE CLEANED DATA IN OTHER NOTEBOOKS

To load this cleaned data with timezone information in your analysis notebooks, use:

```python
import pandas as pd

# Load the cleaned data
df = pd.read_csv('cleaned_date_merged_data.csv')

# Parse the date column
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')

# Verify
print(f"Loaded {len(df):,} rows")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")
print(f"Timezone regions: {df['timezone_region'].unique()}")

# Example: Group by timezone region
region_analysis = df.groupby('timezone_region')['label'].agg(['count', 'sum', 'mean'])
print("\nPhishing rates by region:")
print(region_analysis)

# Example: Filter by specific region
asia_emails = df[df['timezone_region'] == 'APAC']
print(f"\nEmails from APAC: {len(asia_emails):,}")

# Example: Filter by timezone hours (numeric comparisons)
east_asian_tz = df[(df['timezone_hours'] >= 7) & (df['timezone_hours'] <= 9)]
print(f"\nEmails from UTC+7 to UTC+9: {l

In [13]:
# Create Summary Report
print("="*80)
print("TIMEZONE ANALYSIS SUMMARY REPORT")
print("="*80)

summary_report = f"""
TIMEZONE-AWARE EMAIL DATASET SUMMARY
{'='*80}

DATASET OVERVIEW:
- Total emails: {len(df_final):,}
- Date range: {df_final['date'].min()} to {df_final['date'].max()}
- Time span: {(df_final['date'].max() - df_final['date'].min()).days:,} days
- Phishing emails: {(df_final['label'] == 1).sum():,} ({(df_final['label'] == 1).sum()/len(df_final)*100:.1f}%)
- Legitimate emails: {(df_final['label'] == 0).sum():,} ({(df_final['label'] == 0).sum()/len(df_final)*100:.1f}%)

TIMEZONE COVERAGE:
- Emails with timezone info: {df_final['timezone_hours'].notna().sum():,} ({df_final['timezone_hours'].notna().sum()/len(df_final)*100:.1f}%)
- Unique timezone offsets: {df_final['timezone_hours'].nunique()}
- Geographic regions covered: {df_final['timezone_region'].nunique()}

TOP TIMEZONE REGIONS:
"""

for region, count in df_final['timezone_region'].value_counts().head(5).items():
    pct = count / len(df_final) * 100
    phishing_count = df_final[df_final['timezone_region'] == region]['label'].sum()
    phishing_rate = phishing_count / count * 100
    summary_report += f"  {region}: {count:,} emails ({pct:.1f}%) - Phishing rate: {phishing_rate:.1f}%\n"

summary_report += f"""
TEMPORAL PATTERNS:
- Weekend emails: {(df_final['is_weekend'] == 1).sum():,} ({(df_final['is_weekend'] == 1).sum()/len(df_final)*100:.1f}%)
- Weekday emails: {(df_final['is_weekend'] == 0).sum():,} ({(df_final['is_weekend'] == 0).sum()/len(df_final)*100:.1f}%)
- Most active year: {df_final['year'].value_counts().index[0]} ({df_final['year'].value_counts().values[0]:,} emails)
- Peak hour: {df_final['hour'].value_counts().index[0]:02d}:00 ({df_final['hour'].value_counts().values[0]:,} emails)

DATA QUALITY:
- Data retention rate: {len(df_final)/original_count*100:.2f}%
- Missing timezone info: {df_final['timezone_hours'].isna().sum():,}
- Date parsing success: {(df_final['date'].notna().sum() / len(df_final) * 100):.2f}%

{'='*80}
"""

print(summary_report)


TIMEZONE ANALYSIS SUMMARY REPORT

TIMEZONE-AWARE EMAIL DATASET SUMMARY

DATASET OVERVIEW:
- Total emails: 48,803
- Date range: 1990-01-01 17:57:46 to 2022-12-27 10:56:49
- Time span: 12,047 days
- Phishing emails: 27,406 (56.2%)
- Legitimate emails: 21,397 (43.8%)

TIMEZONE COVERAGE:
- Emails with timezone info: 48,803 (100.0%)
- Unique timezone offsets: 40
- Geographic regions covered: 5

TOP TIMEZONE REGIONS:
  Europe/Africa: 27,446 emails (56.2%) - Phishing rate: 59.7%
  Americas: 12,751 emails (26.1%) - Phishing rate: 33.9%
  Middle East/South Asia: 4,776 emails (9.8%) - Phishing rate: 93.2%
  APAC: 2,844 emails (5.8%) - Phishing rate: 76.5%
  Oceania/Pacific: 986 emails (2.0%) - Phishing rate: 7.2%

TEMPORAL PATTERNS:
- Weekend emails: 2,138 (4.4%)
- Weekday emails: 46,665 (95.6%)
- Most active year: 2008 (38,838 emails)
- Peak hour: 08:00 (3,172 emails)

DATA QUALITY:
- Data retention rate: 98.99%
- Missing timezone info: 0
- Date parsing success: 100.00%




In [14]:
df_cleaned.head()

,sender,receiver,date,subject,body,label,urls,timezone_hours,timezone_region,year,month,day,hour,day_of_week,day_name,is_weekend
0,Young Esposito <Young@iworld.de>,user4@gvc.ceas-challenge.cc,2008-08-06 07:31:02,Never agree to be a loser,"Buck up, your troubles caused by small dimensi...",1,1,-7.00,Americas,2008,8,6,7,2,Wednesday,0
1,Mok <ipline's1983@icable.ph>,user2.2@gvc.ceas-challenge.cc,2008-08-06 07:31:03,Befriend Jenna Jameson,\nUpgrade your sex and pleasures with these te...,1,1,-5.00,Americas,2008,8,6,7,2,Wednesday,0
2,Daily Top 10 <Karmandeep-opengevl@universalnet...,user2.9@gvc.ceas-challenge.cc,2008-08-06 16:28:00,CNN.com Daily Top 10,>+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+...,1,1,-12.00,Americas,2008,8,6,16,2,Wednesday,0
3,Michael Parker <ivqrnai@pobox.com>,SpamAssassin Dev <xrh@spamassassin.apache.org>,2008-08-06 07:31:20,Re: svn commit: r619753 - in /spamassassin/tru...,Would anyone object to removing .so from this ...,0,1,-6.00,Americas,2008,8,6,7,2,Wednesday,0
4,Gretchen Suggs <externalsep1@loanofficertool.com>,user2.2@gvc.ceas-challenge.cc,2008-08-06 07:31:21,SpecialPricesPharmMoreinfo,\nWelcomeFastShippingCustomerSupport\nhttp://7...,1,1,-4.00,Europe/Africa,2008,8,6,7,2,Wednesday,0
